In [ ]:
import requests
model_filename = "lid.176.ftz"
r = requests.get(f"https://dl.fbaipublicfiles.com/fasttext/supervised-models/{model_filename}")
open(model_filename, 'wb').write(r.content)

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
lang_model = fasttext.load_model(model_filename)

In [ ]:
lang_model.predict("comment tu t'appelles?")

In [ ]:
lang_model.predict("what's your name?")

In [ ]:
lang_model.predict("wie heißt du?")

In [ ]:
lang_model.predict("comment tu t'appelles?", k=5)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(requests.get("http://crism.maden.org/dunno.html").content)

In [ ]:
import regex as re
ok = not_ok = 0
for p in soup.select("p.translation"):
    text = p.text.strip().replace("\n", " ")
    try:
        correct = re.sub(r"\-.*", "", p.attrs["lang"])
    except:
        correct = "en"
    if correct != 'x':
        l = lang_model.predict(text, k=2)
        # Hauptsprache doppelt so wahrscheinlich wie zweite Wahl?
        if l[1][0] > 2*l[1][1]:
            predict = l[0][0].replace("__label__", "")
            if predict == correct:
                ok += 1
            else:
                print(f"Fehler bei '{text}'")
                print(f"sollte {correct} sein, {predict} vorhergesagt")
                not_ok += 1

print(f"ok = {ok}, not ok = {not_ok}")